# 自然言語処理を用いた質問応答

## 1.1 自然言語処理における深層学習
自然言語処理において深層学習は下記のような様々なタスクで使われる
- 文章を異なる言語に翻訳する機械翻訳
- 重要な情報だけを抽出する自動要約
- 文書と質問を元に回答する機械読解
- 画像に関する質問に応答するシステム
- etc

これらはいずれも世界中の研究者・有名IT企業が精力的に取り組んでいる分野であり、</br>
これらに用いられる最新の手法にはほぼ必ずと言っていいほど深層学習が用いられている</br>
よって、自然言語処理関連の研究・ビジネスをしようとしている人にとって、深層学習・ニューラルネットワークを学ぶことはほぼ必須

なぜ自然言語処理で深層学習がこれほど使われているかというと、</br>
単語をコンピュータで扱うためにはどうしても数値に変換する必要ある</br>
その古典的な方法として、
- One hot vector
- TFIDF

などを「自然言語処理」講座で学んだが、</br>
実際これらは手軽に行えるので手始めに自然言語処理で何かしたい時には最適な手法である</br>
<b>しかし、これらのベクトルには、以下の問題がある</b>

そこでニューラルネットワークのモデルを使うと、誤差逆伝播法によって単語のベクトルを学習できるため、</br>
各単語にわずか数百次元のベクトルを割り当てる（Embeddingと言う）だけでよく、</br>
さらに文脈を考慮しながら単語のベクトルを学習できるので、単語の意味が似ているものは似たようなベクトルが得られるなど、</br>
TFIDFなどに比べて単語の意味を扱えると言った利点がある


## 1.2 Embedding
Embeddingとは日本語で「埋め込み」という意味</br>
深層学習による自然言語処理では、単語という記号をdd次元ベクトル（ddは100〜300程度）に埋め込む、Embeddingという処理を行う</br>
単語を扱うニューラルネットワークのモデルを構築する際、一番始めに行うのがこのEmbedding</br>
tensorflow.kerasではEmbedding Layerというものが用意されており、以下のように使える

ここで必ず指定しなければならない値が、
- input_dim: 語彙数、単語の種類の数
- output_dim: 単語ベクトルの次元
- input_length: 各文の長さ

全単語の単語ベクトルを結合したEmbedding Matrixと呼ばれるものの例が以下</br>
前提として各単語に特有のIDを割り振り、そのIDがEmbedding Matrixの何行目になるのかを指す （ID=0が0行目、ID=iがi行目）</br>
そして各単語のベクトルがEmbedding Matrixの行に相当</br>
Embedding Matrixの横の長さdは単語ベクトルの次元に相当</br>

![](images/Embedding_Matrix.jpeg)

各セルの値はtensorflow.kerasが自動でランダムな値を格納</br>
図のように、多くの場合0行目にはunk、すなわちUnknown（未知語）を割り当てる</br>
unkを使う理由は、扱う語彙を限定してその他のマイナーな単語は全てUnknownとすることでメモリを節約するため</br>
語彙の制限の仕方は、扱うコーパス（文書）に出現する頻度が一定以上のものだけを扱うなどが一般的

- Caution
    - Embeddingへの入力は単語に割り当てたIDからなる行列で、 サイズは（バッチサイズ、文の長さ）である必要がある
      バッチサイズとは、一度に並列して計算を行うデータ（文の）数を表す
    - 文の長さは全てのデータで統一する必要がある
      ここで問題になるのが、文の長さは一定ではないということ
        - そこで恣意的に文の長さをDとし、
            - 長さD以下の文は長さがDになるよう0を追加する
            - 長さD以上の文は長さがDになるよう末尾から単語を削る


In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding


batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ

# 本来は単語をIDに変換する必要がありますが、今回は簡単に入力データを準備しました。
input_data = np.arange(batch_size * seq_length).reshape(batch_size, seq_length)

# modelにEmbeddingを追加
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length))

# input_dataのshapeがどのように変わるのか確認してください。
output = model.predict(input_data)
print(output.shape)

(32, 20, 100)


2021-12-29 22:22:18.153636: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


## 1.3 RNN
RNNとはRecurrent Neural Networkの略称で、日本語で「再帰ニューラルネット」</br>
可変長系列、すなわち任意の長さの入力列を扱うことに優れており、自然言語処理において頻繁に使われる重要な機構</br>
言語以外でも、時系列データであれば使えるので、音声認識など活用の幅は広い

Recurrentは「繰り返しの」という意味</br>
つまりRNNとは「繰り返しの」ニューラルネットワークという意味

![](images/rnn_math.png)

![](images/rnn_image.png)

tensorflow.kerasを使うときにこれらの厳密な定義を覚えておく必要はない</br>
入力列を順番に入力していき、各時刻で隠れ状態ベクトルと出力が得られるということを覚えておくこと</br>
RNNにも他のニューラルネットと同様に複数の層を重ねることができる


## 1.4 LSTM

### 1.4.1 LSTMとは
LSTMとはRNNの一種で、 LSTMはRNNに特有な欠点を補う機構を持っている

- RNN特有の欠点
  RNNは時間方向に深いニューラルネットなので、初期の方に入力した値を考慮してパラメータを学習させるのが難しい
  つまり、長期記憶(long-term memory)が苦手
  感覚的に言うと、RNNは初めの方に入力された要素を「忘れて」しまう

上記の欠点を補うための機構として有名なのがLSTM</br>
LSTMとはLong Short-Term Memoryの略称で、その名の通り長期記憶と短期記憶を可能にするもの</br>
LSTMは1.3章のRNNに「ゲート」という概念を導入したもので、ゲート付きRNNの一種</br>
このゲートによって長期記憶と短期記憶が可能

![](images/lstm_image.png)


### 1.4.2 LSTMの実装
早速tensorflow.kerasでLSTMを実装していく</br>
tensorflow.kerasにはLSTMを簡単に使うことができるモジュールが用意されているため、</br>
数式を意識することなくLSTMを使うことができる</br>

```python
from tensorflow.keras.layers import LSTM
units = 200
lstm = LSTM(units, return_sequences=True)
```

units: LSTMの隠れ状態ベクトルの次元数であり、大抵100から300程度の値</br>
一般に学習すべきパラメータの数は多いほど複雑な現象をモデル化できるが、その分学習させるのが大変（消費メモリが増える、学習時間が長い）

return_sequences: LSTMの出力の形式をどのようにするかを決めるための引数</br>
return_sequencesがTrueなら、LSTMは全ての入力系列に対応する出力系列（隠れ状態ベクトルh_1〜h_T）を出力</br>
return_sequencesがFalseなら、LSTMは最後の時刻TTにおける隠れ状態ベクトルのみを出力</br>
後の章で全ての出力系列を使うことになるので、ここではreturn_sequencesをTrueにしておく

モデルの構築方法は1.2章で学んだEmbeddingと繋げると以下のようになる
```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM

vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(LSTM(lstm_units, return_sequences=True))
```

In [2]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM


batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数

# 今回も簡単に入力データを準備しました。
input_data = np.arange(batch_size * seq_length).reshape(batch_size, seq_length)

# modelにLSTMを追加してください。
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length))
model.add(LSTM(lstm_units, return_sequences=True))


# input_dataのshapeがどのように変わるのか確認してください。
output = model.predict(input_data)
print(output.shape)

(32, 20, 200)


### 1.4.3 BiLSTM
LSTM含めRNNに入力系列x={x_1, ..., x_T}をx_1からx_Tにかけて先頭から順に入力</br>
x_Tからx_1にかけて後ろから順に入力して行くこともできる</br>
さらに2つの入力させる向きを組み合わせた双方向再帰ニューラルネット（bi-directional recurrent neural network）がよく用いられる</br>
利点は、各時刻において先頭から伝播してきた情報と後ろから伝播してきた情報、すなわち入力系列全体の情報を考慮できること</br>
2方向のLSTMを繋げたものをBidirectional LSTM、通称BiLSTMと言う

![](images/BiLSTM_image.png)
2つの向きのRNNを繋げる方法はいくつかあるが</br>
LSTMを引数にすることで簡単に実装できる

```
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
bilstm = Bidirectional(LSTM(units, return_sequences=True), merge_mode='sum')
```

- merge_mode
  2方向のLSTMをどう繋げるかを指定するためのもので、 基本的に{'sum', 'mul', 'concat', 'ave'}の中から選ぶ
    - sum: 要素和
    - mul: 要素積
    - concat: 結合
    - ave: 平均
    - None: 結合せずにlistを返す

![](images/image_merge_mode.png)


In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional


batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length = 20 # 文の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数

# 今回も簡単に入力データを準備しました。
input_data = np.arange(batch_size * seq_length).reshape(batch_size, seq_length)
print(input_data.shape)
# modelにBiLSTMを追加してください。
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length))
model.add(Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='sum'))


# input_dataのshapeがどのように変わるのか確認してください。
output = model.predict(input_data)
print(output.shape)

(32, 20)
(32, 20, 200)


## 1.5 Softmax関数
Softmax関数は活性化関数の一種で、クラス分類を行う際にニューラルネットの一番出力に近い層で使われる

![](images/Softmax.png)

実際にtensorflow.kerasで実装するときは、以下のように、バッチごとにsoftmax関数を適用して使う
```python
from tensorflow.keras.layers import Activation

# xのサイズ: [バッチサイズ、クラス数]
y = Activation('softmax')(x)
# sum(y[0]) = 1, sum(y[1]) = 1, ...
```

これはActivation('softmax')のデフォルトの設定が</br>
入力xxのサイズの最後の要素、クラス数の軸方向にsoftmaxを作用させるというものだから</br>
つまり、xxのサイズが[バッチサイズ、d、クラス数]のように3次元であってもActivation('softmax')を適用可能</br>

note: tensorflow.keras.models.Sequentialを使わずにこのようにモデルを記述する方法をFunctional APIと言う


In [5]:
import numpy as np
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model


x = Input(shape=(20, 5))
# xにsoftmaxを作用させてください
y = Activation('softmax')(x)

model = Model(inputs=x, outputs=y)

sample_input = np.ones((12, 20, 5))
sample_output = model.predict(sample_input)

print(np.sum(sample_output, axis=2))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


### 1.6 Attention
### 1.6.1 Attentionとは
今2つの文章s={s_1, ... , s_N}, t={t_1, ... , t_L}があるとする</br>
ここでは仮にssを質問文とし、ttをそれに対する回答文の候補だとする</br>
この時、機械に自動でtがsに対する回答文として妥当かどうか判断させるにはどのようにしたら良いか？</br>
tだけをいくら眺めても、tが妥当かどうかは分からない</br>
sの情報を参照しつつ、tが妥当かどうか判断する必要がある</br>

<b>そこでAttention Mechanism（注意機構）という機構が役に立つ</b>

これまでの章で文をRNNによって隠れ状態ベクトルに変換できることを学んできた</br>
具体的には2つの別々のRNNを用意し、一方のRNNによってsを隠れ状態ベクトルh^{(s)}={h_1^{(s)},...,h_N^{(s)}}に変換し、</br>
もう一方のRNNによってtを隠れ状態ベクトルh^{(t)}={h_1^{(t)},...,h_L^{(t)}}に変換できる</br>

そこでsの情報を考慮してtの情報を使うために、以下のようにtの各時刻においてsの各時刻の隠れ状態ベクトルを考慮した特徴を計算する</br>
![](images/Attention.png)

図には単方向のRNNの場合を示しましたが、双方向のRNNであってもAttentionは適用可能

![](images/Attention_image.png)

このAttentionという機構は深層学習による自然言語処理では当たり前のように使われる重要な技術で、</br>
機械翻訳や、自動要約、対話の論文で頻繁に登場する</br>
歴史的には機械翻訳に初めて使われて以来その有用性が広く認知されるようになった</br>

また、今回はsの隠れ状態ベクトルの重み付き平均を用いる、Soft Attentionを紹介したが、</br>
ランダムに1つの隠れ状態ベクトルを選択する、Hard Attentionも存在する</br>
さらにそこから派生して、画像認識の分野でも使われることがある</br>
中でもGoogleが発表したAttention is all you needという論文はとても有名


### 1.6.2 Attentionの実装
tensorflow.kerasでAttentionを実装するためには、Mergeレイヤーを使う必要がある
tensorflow.kerasのバージョン2.0以降では前の章まで使っていたSequential ModelをMergeすることができないため、
ここではtensorflow.kerasのFunctional APIを使う。Functional APIの簡単な使い方は以下

Sequential Modelではただ新しいLayerをaddしていくだけでしたが、
Functional APIを使うことでもっと自在に複雑なモデルを組むことができる

```
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import dot, concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model

batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length1 = 20 # 文1の長さ
seq_length2 = 30 # 文2の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数
hidden_dim = 200 # 最終出力のベクトルの次元数

input1 = Input(shape=(seq_length1,))
embed1 = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length1)(input1)
bilstm1 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)

input2 = Input(shape=(seq_length2,))
embed2 = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=seq_length2)(input2)
bilstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)

# 要素ごとの積を計算する
product = dot([bilstm2, bilstm1], axes=2) # productのサイズ：[バッチサイズ、文2の長さ、文1の長さ]

a = Activation('softmax')(product)
c = dot([a, bilstm1], axes=[2, 1])
c_bilstm2 = concatenate([c, bilstm2], axis=2)
h = Dense(hidden_dim, activation='tanh')(c_bilstm2)

model = Model(inputs=[input1, input2], outputs=h)
```

このように各Layerを関数のように使うのでFunctional APIと呼ばれる</br>
また新しく登場したInputレイヤーで指定するshapeにはbatchサイズを入れないよう注意</br>
そしてModelを定義するときは引数にinputsとoutputsを指定する必要があるが</br>
入力や出力が複数ある場合はリストに入れて渡せば大丈夫</br>
そして新しく登場したdot([u, v], axes=2)は、uとvのバッチごとの行列積を計算</br>
指定したaxesにおける次元数はuとvで等しくなければならない</br>
また、dot([u, v], axes=[1,2])とするとuの1次元とvの2次元、のように別々の次元の指定も可能

![](images/Attention_impl.png)




In [7]:
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import dot, concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.models import Model

batch_size = 32 # バッチサイズ
vocab_size = 1000 # 扱う語彙の数
embedding_dim = 100 # 単語ベクトルの次元
seq_length1 = 20 # 文1の長さ
seq_length2 = 30 # 文2の長さ
lstm_units = 200 # LSTMの隠れ状態ベクトルの次元数
hidden_dim = 200 # 最終出力のベクトルの次元数

# 2つのLSTMに共通のEmbeddingLayerを使うため、はじめにEmbeddingLayerを定義します。
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

input1 = Input(shape=(seq_length1,))
embed1 = embedding(input1)
bilstm1 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed1)

input2 = Input(shape=(seq_length2,))
embed2 = embedding(input2)
bilstm2 = Bidirectional(LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed2)

# 要素ごとの積を計算する
product = dot([bilstm2, bilstm1], axes=2) # サイズ：[バッチサイズ、文2の長さ、文1の長さ]

# ここにAttention mechanismを実装してください
a = Activation('softmax')(product)
c = dot([a, bilstm1], axes=[2, 1])
c_bilstm2 = concatenate([c, bilstm2], axis=2)
h = Dense(hidden_dim, activation='tanh')(c_bilstm2)

model = Model(inputs=[input1, input2], outputs=h)

sample_input1 = np.arange(batch_size * seq_length1).reshape(batch_size, seq_length1)
sample_input2 = np.arange(batch_size * seq_length2).reshape(batch_size, seq_length2)

sample_output = model.predict([sample_input1, sample_input2])
print(sample_output.shape)

(32, 30, 200)


2021-12-30 22:27:50.739291: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:689] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


## 1.7 Dropout
Dropoutとは訓練時に変数の一部をランダムに0に設定することによって、汎化性能を上げ、過学習を防ぐための手法

- 過学習とは
    - ニューラルネットなどのモデルで教師あり学習を行う場合、しばしば訓練データに適合しすぎて、
      検証データでのパフォーマンスが訓練データに比べて著しく下がる「過学習」を起こしてしまう
- 汎化性能とは
    - 訓練データで過学習することなく、訓練データと検証データに関わらず一般的に高いパフォーマンスができることを「汎化性能」が高いと言う
      実際に使うときは、変数のうち0に設定する割合を0から1の間の値で設定して、Dropoutレイヤーを追加する

```python
# Sequentialモデルを使う場合
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout

model = Sequential()
...
model.add(Dropout(0.3))

# Functional APIを使う場合
from tensorflow.keras.layers import Dropout
y = Dropout(0.3)(x)
```

In [8]:
import numpy as np
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Model

batch_size = 32  # バッチサイズ
vocab_size = 1000  # 扱う語彙の数
embedding_dim = 100  # 単語ベクトルの次元
seq_length = 20  # 文1の長さ
lstm_units = 200  # LSTMの隠れ状態ベクトルの次元数

input = Input(shape=(seq_length,))

embed = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  input_length=seq_length)(input)

bilstm = Bidirectional(
    LSTM(lstm_units, return_sequences=True), merge_mode='concat')(embed)

output = Dropout(0.3)(bilstm)

model = Model(inputs=input, outputs=output)

sample_input = np.arange(
    batch_size * seq_length).reshape(batch_size, seq_length)

sample_output = model.predict(sample_input)

print(sample_output.shape)

(32, 20, 400)


# 回答文選択システムの実装

## 2.1 回答文選択システム
実践編では、基礎編で学んだことを活かして回答文選択システムを実装</br>
質問文に対して、回答文の候補がいくつか与えられて、その中から正しい回答文を自動で選択するシステム</br>
用いるデータセットはAllen AIのTextbook Question Answeringというもの

1. 分かち書き
2. 文字の正規化
3. 単語のID化
4. (自然言語処理で深層学習を使う場合) Padding(= 入力の全ての文の長さを統一)


### 2.2.1 正規化・分かち書き
